In [9]:
import PIL, torch, torchvision
from torch.autograd import Variable
from PIL import Image

In [24]:
# Convert images to 224x224x3
# image_file = 'golden_retriever.jpeg'
# image = Image.open(image_file)

# w = 224
# h = 224

# out = image.resize((w, h), Image.ANTIALIAS)
# out.save('test.jpg')


# Load and normalize the image
img = torchvision.transforms.ToTensor()(PIL.Image.open('test.jpg'))
img = img.view(1, img.size(0), img.size(1), img.size(2))
img = 0.5 + 0.5 * (img - img.mean()) / img.std()

# Load an already trained network and compute its prediction
alexnet = torchvision.models.alexnet(pretrained=True)
alexnet.eval()

output = alexnet(Variable(img))

# Print the classes
scores, indexes = output.data.view(-1).sort(descending=True)

class_names = eval(open('imagenet1000_clsid_to_human.txt', 'r').read())

for k in range(15):
    print('#{:d} ({:.02f}) {:s}'.format(k, scores[k], class_names[int(indexes[k])]))

torch.Size([3, 224, 224])
torch.Size([1, 3, 224, 224])
#0 (13.66) golden retriever
#1 (13.50) Labrador retriever
#2 (12.15) kuvasz
#3 (10.66) Chesapeake Bay retriever
#4 (10.60) ice bear, polar bear, Ursus Maritimus, Thalarctos maritimus
#5 (10.59) cocker spaniel, English cocker spaniel, cocker
#6 (10.35) Great Pyrenees
#7 (9.09) clumber, clumber spaniel
#8 (9.05) standard poodle
#9 (8.76) Tibetan terrier, chrysanthemum dog
#10 (8.62) otterhound, otter hound
#11 (8.53) chow, chow chow
#12 (8.47) beagle
#13 (8.41) soft-coated wheaten terrier
#14 (8.41) English setter


In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch import nn

train_transforms = transforms.Compose(
[
    transforms.RandomCrop(28, padding=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=(33.32, ), std=(78.56, ))
])

train_loader = DataLoader(datasets.MNIST(root='./data', train=True, download=True, transform=train_transforms),
                         batch_size=100,
                         num_workers=4,
                         shuffle=True,
                         pin_memory=torch.cuda.is_available())


"""
This method is only useful for large datasets, since it loads a small part of data to CPU->GPU.
Hence, with small datasets like MNIST or CIFAR10, this will take more times because of loading from CPU to GPU
"""
for e in range(nb_epochs) :
    for input, target in iter(train_loader) :
        if torch.cuda.is_available() :
        input, target = input.cuda(), target.cuda()
    input, target = Variable(input), Variable(target)
    
    output = model(input)
    loss = criterion(output, target)
    model.zero_grad()
    loss.backward()
    optimizer.step()

In [9]:
"""
REUSING A NETWORK & FINE-TUNING IT
• Dataset: CIFAR10
• the first layer of an [already trained] AlexNet,
• several resnet blocks, stored in a nn.ModuleList and each combining nn.Conv2d , nn.BatchNorm2d , and nn.ReLU,
• a final channel-wise averaging, using nn.AvgPool2d, and
• a final fully connected linear layer nn.Linear .

During training, we keep the AlexNet features frozen for a few epochs.
This is done by setting requires_grad of the related Parameters to False.
"""
import torchvision
import torch
from torch import Tensor, nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
from torch.autograd import Variable


data_dir = './data/cifar10'

num_workers = 4
batch_size = 64

transform = torchvision.transforms.ToTensor()

train_set = torchvision.datasets.CIFAR10(root = data_dir, train=True, download=False, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers = num_workers)

test_set = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=False, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)


def make_resnet_block(nb_channels, kernel_size=3):
    return nn.Sequential(nn.Conv2d(nb_channels, nb_channels, kernel_size=kernel_size, padding=(kernel_size-1)//2),
                        nn.BatchNorm2d(nb_channels),
                        nn.ReLU(inplace=True),
                        nn.Conv2d(nb_channels, nb_channels, kernel_size=kernel_size, padding=(kernel_size-1)//2),
                        nn.BatchNorm2d(nb_channels))


class Monster(nn.Module):
    def __init__(self, nb_residual_blocks, nb_channels):
        super(Monster, self).__init__()
        nb_alexnet_channels = 64
        alexnet_feature_map_size = 7 # For 32x32 ( e.g.CIFAR )
        alexnet = torchvision.models.alexnet(pretrained = True)
        # Conv2d (3 , 64 , kernel_size =(11 , 11) , stride =(4 , 4) , padding =(2 , 2) )
        self.features = nn.Sequential(alexnet.features[0],
                                      nn.ReLU (inplace = True))
        self.converter = nn.Sequential(nn.Conv2d(nb_alexnet_channels, nb_channels, kernel_size=3, padding=1),
                                       nn.ReLU(inplace=True))
        self.resnet_blocks = nn.ModuleList()
        for k in range (nb_residual_blocks):
            self.resnet_blocks.append(make_resnet_block(nb_channels, 3))
            self.final_average = nn.AvgPool2d(alexnet_feature_map_size)
            self.fc = nn.Linear(nb_channels, 10)
            
            
    def freeze_features(self, q):
        for p in self.features.parameters():
            # If frozen ( q == True ) we do NOT need the gradient
            p.requires_grad = not q

    def forward(self, x):
        x = self.features(x)
        x = self.converter(x)
        for b in self.resnet_blocks:
            x = x + b(x)
        x = self.final_average(x).view(x.size(0), -1)
        x = self.fc(x)
        return x



nb_epochs = 100
nb_epochs_frozen_features = nb_epochs // 2
nb_residual_blocks = 16
nb_channels = 64

model, criterion = Monster(nb_residual_blocks, nb_channels), nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
    
optimizer = optim.SGD(model.parameters(), lr = 1e-2)

model.train(True)

for e in range(nb_epochs):
    model.freeze_features(e < nb_epochs_frozen_features)
    
    acc_loss = 0.0
    
    for input, target in iter(train_loader):
        if torch.cuda.is_available():
            input, target = input.cuda(), target.cuda()
            
        input, target = Variable(input), Variable(target)
        
        output = model(input)
        loss = criterion(output, target)
        acc_loss += loss.data[0]
        
        model.zero_grad()
        loss.backward()
        optimizer.step()
        
    print (e, acc_loss)
    
    
nb_test_errors, nb_test_samples = 0, 0

model.train(False)

for input, target in iter(test_loader):
    if torch.cuda.is_available():
        input = input.cuda()
        target = target.cuda()
        
    input = Variable(input)
    
    output = model(input)
    wta = torch.max(output.data, 1)[1].view(-1)
    
    for i in range(target.size(0)):
        nb_test_samples += 1
        if wta [i] != target[i]:
            nb_test_errors += 1
            
print ('test_error {:.02f}% ({:d}/{:d})'.format(100 * nb_test_errors / nb_test_samples,
                                                       nb_test_errors,
                                                       nb_test_samples))

Files already downloaded and verified
Files already downloaded and verified


/home/duytue/miniconda3/envs/deeplearn/lib/python3.6/site-packages/ipykernel_launcher.py:104: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


0 tensor(973.2639)
1 tensor(741.4677)
2 tensor(650.1761)


Process Process-17:
Process Process-18:
Process Process-19:
Traceback (most recent call last):
Process Process-20:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/duytue/miniconda3/envs/deeplearn/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/duytue/miniconda3/envs/deeplearn/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/duytue/miniconda3/envs/deeplearn/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/duytue/miniconda3/envs/deeplearn/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/duytue/miniconda3/envs/deeplearn/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/duytue/miniconda3/envs/deeplearn/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 52, in _wor

KeyboardInterrupt: 